In [ ]:
using Random
using DataFrames
using CSV

In [ ]:
function build_lattice_df(Lx,Ly)
    
    N = Lx*Ly
    interactions = DataFrame(["$spin" => [] for spin in 1:N])    
    for spin in 1:N
        up = mod(spin-1-Lx,N)
        down = mod(spin-1+Lx,N)
        right = Int(((spin-1)-((spin-1)%Lx)) + mod(((spin-1)%Lx)+1,Lx))
        left = Int(((spin-1)-((spin-1)%Lx)) + mod(((spin-1)%Lx)-1,Lx))
        push!(interactions[!,"$spin"],[up+1,down+1,left+1,right+1])
    end
    
    return interactions
end

In [ ]:
function Calculate_Ising_Energy(config,interactions,J)
    
    N = length(config)
    energy = 0
    
    for spin in (1:N)
        neighbors = interactions[!,"$spin"][1,1]
        energy_contributions = 0.5*-1*J*config[spin].*config[neighbors]
        energy += sum(energy_contributions)
    end
    
    return energy
    
end

In [ ]:
function build_deltaE_df(beta)
    
    possible_sum_neighbors = collect(-4:2:4)
    spins = [1,-1]
    deltaEs = DataFrame(["$spin,$poss" => [] for poss in possible_sum_neighbors for spin in spins])    
    for sum_neighbors in possible_sum_neighbors
        spinup_deltaE = 2*sum_neighbors
        spindown_deltaE = -2*sum_neighbors 
        push!(deltaEs[!,"1,$sum_neighbors"],spinup_deltaE)
        push!(deltaEs[!,"-1,$sum_neighbors"],spindown_deltaE)
    end
    
    return deltaEs
end

In [ ]:
function Metropolis_update_function(config,energy,interactions,beta,J,deltaE_vals)
    
    rand_spin_flip_index = rand(1:length(config),1)[1]
    new_config = copy(config)

    spin = Int(config[rand_spin_flip_index])
    neighbors = interactions[!,"$rand_spin_flip_index"][1,1]
    sum_neighbors = Int(sum(config[neighbors]))
    deltaE = deltaE_vals[!,"$spin,$sum_neighbors"][1]
    exp_beta_deltaE = exp(-1*beta*deltaE)
    rand_value = rand(Float64,1)[1]
    
    if exp_beta_deltaE > rand_value
        new_config[rand_spin_flip_index] = -1*config[rand_spin_flip_index]
    else
        deltaE = 0.
    end
    
    return new_config, deltaE
end

In [ ]:
function Ising_cluster_update(config,energy,interactions,beta,J,energy_func)
    # angle defining flipping axis
    N = length(config)
    cluster = []
    spins_to_check = []
    initial_spin = rand(1:N)
    prob = 1-exp(-(2*beta))
    append!(cluster, initial_spin)
    append!(spins_to_check, initial_spin)
    while !isempty(spins_to_check)
        # check all neighbours of spins_to_check
        site = spins_to_check[1]
        spin_val = config[site]
        neighbors = interactions[!,"$site"][1,1]
        neighbors_not_in_cluster = setdiff(neighbors,cluster)
        for i in neighbors_not_in_cluster
            spin_neighbor = config[i]
            if spin_neighbor == spin_val
                if prob > rand()
                    append!(cluster, i)
                    append!(spins_to_check, i)
                end
            end
        end
        #remove the one we just checked
        popfirst!(spins_to_check)
    end
    
    for i in cluster
        config[i] = -1*config[i]
    end
    
    new_energy = energy_func(config,interactions,J)
    deltaE = new_energy-energy
    
    return config, deltaE
end

In [ ]:
function MonteCarlo_Ising(beta,J,Lx,Ly,warmup_steps,steps,steps_per_step,build_lattice_function,build_update_probs,update_function,energy_function)
    
    # system information
    N = Lx*Ly
    interactions = build_lattice_function(Lx,Ly)
    deltaEs = build_deltaE_df(beta)
    initial_config = rand((-1,1),N)
    
    # initialize dataframes
    fulldf = DataFrame(:Es => Float64[],:avE => Float64[])
    maindf = DataFrame(:avenergy => Float64[],:avenergy2 => Float64[],:rawMs => Float64[],:avM => Float64[],:avM2 => Float64[])
    
    # warmup steps - track Es and Ms but not in averages
    config = initial_config
    energy = energy_function(config,interactions,J)
    M = 0.
    fullsumE = 0
    for step in (1:warmup_steps)
        for inner_steps in (1:steps_per_step)
            new_config, deltaE = update_function(config,energy,interactions,beta,J,energy_function)
#             new_config,deltaE = update_function(config,energy,interactions,beta,J,deltaEs)
            config = copy(new_config)
            new_energy = energy+deltaE
            energy = new_energy
            fullsumE = fullsumE+energy
            M = sum(new_config)
        end
        push!(fulldf,(energy,fullsumE/(step*steps_per_step)))
    end
    
    # begin MC steps and observable tracking
    sumE = 0
    sumE2 = 0
    sumM = 0
    sumM2 = 0
    for step in (1:steps)
        for inner_step in (1:steps_per_step)
            new_config,deltaE = update_function(config,energy,interactions,beta,J,energy_function)
#             new_config,deltaE = update_function(config,energy,interactions,beta,J,deltaEs)
            config = copy(new_config)
            new_energy = energy+deltaE
            energy = new_energy
            fullsumE = fullsumE+energy
        end
        M = sum(config)
        push!(fulldf,(energy,fullsumE/((step*steps_per_step)+warmup_steps)))
        absM = abs(M)
        sumE = sumE+energy
        sumE2 = sumE2+energy^2
        sumM = sumM+absM
        sumM2 = sumM2+absM^2
        push!(maindf, (sumE/step,sumE2/step,M,sumM/step,sumM2/step))
    end
        
    return maindf,fulldf
end

# Q 1 a

In [ ]:
Ts = [1.0,1/3]
Ls = [16]
J = 1
N_chains = 5
n_steps = 1000000
warmup = Int(n_steps/10)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=3)
        for chain in 1:N_chains
            println("chain #",chain)
            @time begin
            avgs,full = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,1,build_lattice_df,build_deltaE_df,Metropolis_update_function,Calculate_Ising_Energy);
            end
            CSV.write("./data/q1a/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
            CSV.write("./data/q1a/L_$(L)/fullavgvals_T_$(T)_chain$(chain).csv",  full, writeheader=true)
        end
    end
end

In [ ]:
Ts = [1.0,1/3]
Ls = [16]
J = 1
N_chains = 5
n_steps = 1000000
warmup = Int(n_steps/10)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=3)
        for chain in 2:N_chains
            println("chain #",chain)
            @time begin
            avgs,full = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,(L^2)/2,build_lattice_df,build_deltaE_df,Metropolis_update_function,Calculate_Ising_Energy);
            end
            CSV.write("./data/q1a/L_$(L)/avgvals2_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
            CSV.write("./data/q1a/L_$(L)/allvals2_T_$(T)_chain$(chain).csv",  full, writeheader=true)
        end
    end
end

# Q 1 b, c, d

Need to continue runs for L = 32

In [ ]:
Ts = 2.0:0.05:4.0
Ls = [32]
J = 1
N_chains = 10
n_steps = 5000
warmup = Int(n_steps/5)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=2)
        for chain in 1:N_chains
            println("chain #",chain)
            avgs,full = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,1,build_lattice_df,build_deltaE_df,Ising_cluster_update,Calculate_Ising_Energy);
#             CSV.write("./data/q1b/L_$(L)/allvals_T_$(T)_chain$(chain).csv",  full, writeheader=true)
            CSV.write("./data/q1b_cluster/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
        end
    end
end

Finer T grid for smaller system sizes

Need to finish for L = 24!!!! Run L = 32??

In [ ]:
Ts = 1.56:0.01:4.0
Ls = [24]
J = 1
N_chains = 10
n_steps = 5000
warmup = Int(n_steps/5)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=2)
        for chain in 1:N_chains
            println("chain #",chain)
            avgs,full = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,1,build_lattice_df,build_deltaE_df,Ising_cluster_update,Calculate_Ising_Energy);
#             CSV.write("./data/q1b/L_$(L)/allvals_T_$(T)_chain$(chain).csv",  full, writeheader=true)
            CSV.write("./data/q1b_cluster/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
        end
    end
end